In [13]:
# we load the things!

from ngsolve import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
from netgen.geom2d import SplineGeometry
from ngsolve.krylovspace import GMResSolver

from ngsolve.krylovspace import CGSolver

In [14]:
# Geometry

def geometry(R, PMLw, r, h_max, curve_order):

    sphere_R = Sphere(Pnt(0, 0, 0), R)                  # Largest sphere
    sphere_Rr = Sphere(Pnt(0, 0, 0), R-PMLw)             # middle sphere
    sphere_rr = Sphere(Pnt(0, 0, 0), r)                 # smallest sphere

    sphere_R.faces.name = "outer"
    sphere_rr.faces.name = "inner"

    PML = sphere_R - sphere_Rr
    vacuum = sphere_Rr - sphere_rr

    PML.name = "PML"
    vacuum.name = "vacuum"
    

    domain = Glue([PML, vacuum])

    geo = OCCGeometry(domain, dim=3)                    # cast to open cascade type
    mesh = Mesh(geo.GenerateMesh(maxh=h_max))           # mesh open cascade object
    mesh.Curve(curve_order)                             # curve mesh to capture sphere curvature
    mesh.SetPML(pml.Radial(rad=R-PMLw, alpha=1j, origin=(0,0,0)), "PML") # set PML Domain

    return mesh

In [15]:
R, r, PMLwidth, hmax, curve_order = 1, 0.1, 0.25, 0.5, 5


Lam = 0.65    # radar wavelength Lambda 

k = 2*pi / Lam # Wavenumber k
mesh = geometry(R, PMLwidth, r, hmax, curve_order)

fes = HCurl(mesh, order= 5, type1=False, complex=True, dirichlet="outer")

E, Ep = fes.TnT() 

n, t = specialcf.normal(3), specialcf.tangential(3)

In [16]:
p = CF((0,0,1))     # Poynting vector
pol = CF((1,0,0))   # Polarisation
coord_vec = CF((x,y,z))

phase = exp(1j * k * p * coord_vec)
#phase = exp(1 * k * p * coord_vec)
E_inc = pol * phase 

f = CF((0,0,0))

In [17]:
a = BilinearForm(fes, symmetric=False)
a += curl(E) * curl(Ep) * dx
a += - k**2 * Ep * E * dx
#a += 1e-6 * Ep * E * dx # regularization

a += - 1j * k * E.Trace() * Ep.Trace() * ds("inner")
#a += - 1 * k * E.Trace() * Ep.Trace() * ds("inner")

l = LinearForm(fes)
l += f * Ep * dx
l += - 1j * k * Cross(n, E_inc) * Ep.Trace() * ds("inner")
#l += - 1 * k * Cross(n, E_inc) * Ep.Trace() * ds("inner")

In [18]:
# Solve Maxwell Helmholtz Scattering
 
useGMRes = True

# print("coming till here..")
with TaskManager(): 
    if (useGMRes == False):
        a.Assemble()
        l.Assemble()
        sol = GridFunction(fes)
        res = l.vec-a.mat * sol.vec
        inv = a.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky")   # direct solver (small demo)
        sol.vec.data += inv * res
    else:
        #with TaskManager():
        a.Assemble()
        l.Assemble()
        gfu = GridFunction(fes)
        blocks = fes.CreateSmoothingBlocks()
        preBJ = a.mat.CreateBlockSmoother(blocks) #Block Jacobi Preconditioner
        #GMResSolver(A=a.mat, x= gfu.vec, b=l.vec, pre=preBJ, printrates="\r", maxsteps = 1000, tol=1e-6)
        solvers.GMRes(A=a.mat, x= gfu.vec, b=l.vec, pre=preBJ, printrates="\r", maxsteps = 1000, tol=1e-6)
#print(F.vec)


hcurl smoothingblocks, SmoothingType = 2
GMRes converged in 904 iterations to residual 9.931795169968783e-07


In [19]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'spe…

BaseWebGuiScene

In [20]:
# Solve Maxwell Helmholtz Scattering
 
#useGMRes = True


# Hiptmair Xu type preconditioner
#print("before problem")
#preHX = Preconditioner(a, "hcurlamg")
#preBDDC = Preconditioner(a, "bddc")
# print("after problem")
# with TaskManager():
#     a.Assemble()
#     print("after assembling a")
#     l.Assemble()
#     print("after assembling l")
#     gfu = GridFunction(fes)
#     print("right before solving")
#     inv = CGSolver(a.mat, preBDDC.mat, plotrates=True, maxiter=200)
#     gfu.vec[:] = inv*l.vec

# print("coming till here..")
# with TaskManager(): 
#     if (useGMRes == False):
#         a.Assemble()
#         l.Assemble()
#         sol = GridFunction(fes)
#         res = l.vec-a.mat * sol.vec
#         inv = a.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky")   # direct solver (small demo)
#         sol.vec.data += inv * res
#     else:
#         #with TaskManager():
#         a.Assemble()
#         l.Assemble()
#         gfu = GridFunction(fes)
#         # blocks = fes.CreateSmoothingBlocks()
#         # preBJ = a.mat.CreateBlockSmoother(blocks)
#         GMRes(A =a.mat,x= gfu.vec, b=l.vec,pre = preHX.mat,  printrates="\r", maxsteps = 1000, tol=1e-6)
# #print(F.vec)


In [21]:
Draw(gfu, mesh)

WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'spe…

BaseWebGuiScene